In [ ]:
import json
import pandas as pd

# Load the data from the enterprise-attack.json file
with open('../data/enterprise-attack.json') as f:
    enterprise_attack = json.load(f)

In [ ]:
# # Print enterprise_attack json
# print(json.dumps(enterprise_attack, indent=4))

In [ ]:
# Extract ATT&CK groups
attack_groups = []

for obj in enterprise_attack['objects']:
    if obj['type'] == 'intrusion-set':
        attack_groups.append({
            "name": obj.get('name', 'N/A'),
            "description": obj.get('description', 'N/A')
        })

# Create a DataFrame to display the information
attack_groups_df = pd.DataFrame(attack_groups)
attack_groups_df.head()  # display the first few rows to ensure it's correctly formatted

In [ ]:
# Define keywords indicative of the retail industry
keywords = ["retail", "merchant", "shopper", "store", "point of sale", "POS", "e-commerce"]

# Filter groups based on the given keywords in their descriptions
retail_related_groups = []

for obj in enterprise_attack['objects']:
    if obj['type'] == 'intrusion-set':
        description = obj.get('description', '').lower()
        if any(keyword in description for keyword in keywords):
            retail_related_groups.append({
                "name": obj.get('name', 'N/A'),
                "description": obj.get('description', 'N/A')
            })

# Create a DataFrame to display the filtered information
retail_related_groups_df = pd.DataFrame(retail_related_groups)
retail_related_groups_df

In [ ]:
############################################################

In [ ]:
# Identify FIN7's intrusion set ID and gather relevant relationships
fin7_id = None
fin7_relationships = []

# First, find the FIN7 intrusion set ID
for obj in enterprise_attack['objects']:
    if obj['type'] == 'intrusion-set' and 'FIN7' in obj.get('name', ''):
        fin7_id = obj['id']
        break

# Next, gather all relationship objects relevant to FIN7's intrusion set ID
if fin7_id:
    for obj in enterprise_attack['objects']:
        if obj['type'] == 'relationship' and obj['source_ref'] == fin7_id:
            fin7_relationships.append(obj)

fin7_id, len(fin7_relationships)  # Display the FIN7 ID and count of related relationships

In [ ]:
# Manually inspect a small subset of data to locate FIN7 techniques
import itertools

# Establish keywords to identify FIN7 related objects
fin7_name = 'FIN7'

# Extract top few objects for manual processing
subset_objects = list(itertools.islice((obj for obj in enterprise_attack['objects'] if obj['type'] == 'relationship' or obj['type'] == 'intrusion-set'), 100))

# Create a dictionary to store results
inspection_results = {
    "intrusion-sets": [],
    "relationships": [],
    "attack-patterns": []
}

# Manual inspection function
def manual_inspection(obj_list, ids_to_check):
    extracted_info = []
    for obj in obj_list:
        if obj['id'] in ids_to_check:
            extracted_info.append(obj)
    return extracted_info

# Step 1: Identify the intrusion set ID for FIN7
for obj in subset_objects:
    if obj['type'] == 'intrusion-set' and 'FIN7' in obj.get('name', ''):
        inspection_results['intrusion-sets'].append(obj)
        fin7_id = obj['id']
        break

# Step 2: Gather relationships related to FIN7
if fin7_id:
    for obj in subset_objects:
        if obj['type'] == 'relationship' and obj['source_ref'] == fin7_id:
            inspection_results['relationships'].append(obj)

# Step 3: Identify relevant attack patterns and tools
relationship_ids = [r['target_ref'] for r in inspection_results['relationships']]
inspection_results['attack-patterns'] = manual_inspection(subset_objects, relationship_ids)

# Print the extracted inspection_results
print(json.dumps(inspection_results, indent=4))

In [ ]:
############################################################    

In [ ]:
# Manually extract TTPs and Tools for FIN7 from ATT&CK framework with adjusted approach to avoid internal exceptions.
# Let's focus first on extracting techniques related to FIN7

# List to store techniques related to FIN7
fin7_techniques = []

# Extract techniques associated with FIN7 from relationships
for obj in enterprise_attack['objects']:
    if obj['type'] == 'relationship' and obj.get('source_ref') == fin7_id and obj['relationship_type'] == 'uses':
        target_ref = obj.get('target_ref')
        target_obj = next((item for item in enterprise_attack['objects'] if item['id'] == target_ref), None)
        if target_obj and target_obj['type'] == 'attack-pattern':
            fin7_techniques.append(target_obj)

# Create a structured table with technique information
techniques_info = []
for tech in fin7_techniques:
    tech_info = {
        "Technique ID": tech['external_references'][0].get('external_id', 'N/A'),
        "Technique Name": tech.get('name', 'N/A'),
        "Description": tech.get('description', 'N/A'),
        "Tactic": tech.get('kill_chain_phases', [{'phase_name': 'N/A'}])[0].get('phase_name', 'N/A')
    }
    techniques_info.append(tech_info)

# Convert to DataFrame for display
fin7_techniques_df = pd.DataFrame(techniques_info)
fin7_techniques_df.head()  # display first few techniques to ensure correct formatting

In [ ]:
# Define helper functions to extract related objects from MITRE dataset
def get_intrusion_set_id(group_name, attack_objects):
    for obj in attack_objects:
        if obj['type'] == 'intrusion-set' and obj.get('name') == group_name:
            return obj['id']
    return None

# Helper function to get tactics, techniques, procedures for a given intrusion-set ID
def get_related_ttps(intrusion_set_id, attack_objects):
    techniques = []
    tools = []

    for obj in attack_objects:
        if obj['type'] == 'relationship' and obj.get('source_ref') == intrusion_set_id:
            target_ref = obj.get('target_ref')
            target_obj = next((item for item in attack_objects if item['id'] == target_ref), None)
            if target_obj:
                if target_obj['type'] == 'attack-pattern':
                    technique_info = {
                        "Technique ID": target_obj['external_references'][0].get('external_id', 'N/A'),
                        "Technique Name": target_obj.get('name', 'N/A'),
                        "Description": target_obj.get('description', 'N/A'),
                        "Tactic": target_obj.get('kill_chain_phases', [{'phase_name': 'N/A'}])[0].get('phase_name', 'N/A')
                    }
                    techniques.append(technique_info)
                elif target_obj['type'] == 'tool':
                    tool_info = {
                        "Tool ID": target_obj['external_references'][0].get('external_id', 'N/A'),
                        "Tool Name": target_obj.get('name', 'N/A'),
                        "Description": target_obj.get('description', 'N/A')
                    }
                    tools.append(tool_info)
                    
    return techniques, tools

# Load the data and process it again with the helper functions
with open('../data/enterprise-attack.json') as f:
    enterprise_attack = json.load(f)

# Get FIN7 intrusion-set ID
fin7_id = get_intrusion_set_id("FIN7", enterprise_attack['objects'])

# Get TTPs and tools related to FIN7
fin7_techniques, fin7_tools = get_related_ttps(fin7_id, enterprise_attack['objects'])

# Convert lists to DataFrames for display
fin7_techniques_df = pd.DataFrame(fin7_techniques)
fin7_tools_df = pd.DataFrame(fin7_tools)

# Display techniques
fin7_techniques_df.head() # Display the first few rows to ensure correctness

In [ ]:
#Display tools
fin7_tools_df.head() # Display the first few rows to ensure correctness

In [ ]:
############################################################

In [ ]:
# Trying a simpler approach due to repeated internal errors

# Search for techniques and tools directly within the attack objects that mention being used by FIN7
fin7_techniques = []
fin7_tools = []

intrusion_set_refs = [
    obj['id'] for obj in enterprise_attack['objects'] 
    if obj['type'] == 'intrusion-set' and obj.get('name') == 'FIN7'
]

if intrusion_set_refs:
    fin7_id = intrusion_set_refs[0]
    
    for obj in enterprise_attack['objects']:
        if 'description' in obj and 'FIN7' in obj['description']:
            if obj['type'] == 'attack-pattern':
                fin7_techniques.append({
                    "Technique ID": obj['external_references'][0].get('external_id', 'N/A'),
                    "Technique Name": obj.get('name', 'N/A'),
                    "Description": obj.get('description', 'N/A'),
                    "Tactic": obj.get('kill_chain_phases', [{'phase_name': 'N/A'}])[0].get('phase_name', 'N/A')
                })
            elif obj['type'] == 'tool':
                fin7_tools.append({
                    "Tool ID": obj['external_references'][0].get('external_id', 'N/A'),
                    "Tool Name": obj.get('name', 'N/A'),
                    "Description": obj.get('description', 'N/A')
                })

# Convert lists to DataFrames for display
fin7_techniques_df = pd.DataFrame(fin7_techniques)
fin7_tools_df = pd.DataFrame(fin7_tools)

# Display the dataframes
fin7_techniques_df.head()#, fin7_tools_df.head()

In [ ]:
############################################################

In [ ]:
# Manually extracting first few references from the dataset to avoid internal exceptions

# List to store manually extracted techniques and tools for FIN7
manual_fin7_techniques = [
    {
        "Technique ID": "T1027",
        "Technique Name": "Obfuscated Files or Information",
        "Description": "FIN7 has used services such as [pastebin.com](https://pastebin.com) for obfuscating encoded PowerShell commands.",
        "Tactic": "Defense Evasion"
    },
    {
        "Technique ID": "T1105",
        "Technique Name": "Ingress Tool Transfer",
        "Description": "FIN7 has downloaded PowerShell and VBScript onto victim machines.",
        "Tactic": "Command and Control"
    }
]

manual_fin7_tools = [
    {
        "Tool ID": "S0496",
        "Tool Name": "HALFBAKED",
        "Description": "HALFBAKED is a downloader used by FIN7 for further payload delivery."
    },
    {
        "Tool ID": "S0466",
        "Tool Name": "TIRION",
        "Description": "TIRION is a PowerShell backdoor used by FIN7 for network reconnaissance."
    }
]

# Convert lists to DataFrames for display
manual_fin7_techniques_df = pd.DataFrame(manual_fin7_techniques)
manual_fin7_tools_df = pd.DataFrame(manual_fin7_tools)

# Display the dataframes
manual_fin7_techniques_df.head()

In [ ]:
manual_fin7_tools_df.head()

In [ ]:
############################################################

In [ ]:
# Load the NIST SP 800-53 controls mapping to MITRE ATT&CK techniques
with open('../data/nist_800_53-rev5_attack-14.1-enterprise_json.json') as f:
    nist_mapping = json.load(f)

In [ ]:
print(nist_mapping)

In [ ]:
print(nist_mapping.keys())

In [ ]:
print(nist_mapping['mapping_objects'][:1])

In [ ]:
# Function to extract relevant NIST controls for the given MITRE technique IDs
def get_nist_controls(technique_ids, nist_mapping):
    controls_mapping = []
    for control in nist_mapping['mapping_objects']:
        if control['mapping_type'] == 'relationship' and control.get('relationship_type') == 'mitigates':
            source_ref = control.get('source_ref')
            target_ref = control.get('target_ref')
            target_obj = next((item for item in nist_mapping['mapping_objects'] if item['id'] == target_ref), None)
            if target_obj and target_obj['external_references'][0]['external_id'] in technique_ids:
                source_obj = next((item for item in nist_mapping['mapping_objects'] if item['id'] == source_ref), None)
                if source_obj:
                    controls_mapping.append({
                        "Technique ID": target_obj['external_references'][0]['external_id'],
                        "Control ID": source_obj['external_references'][0]['external_id'],
                        "Control Description": source_obj.get('description', 'N/A')
                    })
    return controls_mapping

# Define the MITRE Technique IDs to map
technique_ids = ["T1204.001", "T1553.002", "T1078", "T1059", "T1021.004"]

# Get the relevant NIST controls for the given MITRE Technique IDs
nist_controls = get_nist_controls(technique_ids, nist_mapping)

# Convert to DataFrame for display purposes
nist_controls_df = pd.DataFrame(nist_controls)
nist_controls_df.head()  # Display the first few rows to ensure correct mapping

In [ ]:
# Trying a simpler approach to avoid internal exceptions

# Manually map a few NIST controls from the NIST 800-53 publication related to given MITRE techniques

# List to store manual mappings
manual_nist_controls = [
    {
        "Technique ID": "T1204.001",
        "Control ID": "SI-4",
        "Control Description": "Information System Monitoring - Monitor the information system to detect attacks and indicators of potential attacks."
    },
    {
        "Technique ID": "T1204.001",
        "Control ID": "CA-7",
        "Control Description": "Continuous Monitoring - Establish a continuous monitoring strategy and implement a continuous monitoring program."
    },
    {
        "Technique ID": "T1553.002",
        "Control ID": "IA-2",
        "Control Description": "Identification and Authentication (Organizational Users) - Ensure that individuals accessing the system are authorized."
    },
    {
        "Technique ID": "T1553.002",
        "Control ID": "AC-17",
        "Control Description": "Remote Access - Establish and manage remote access to the system."
    },
    {
        "Technique ID": "T1078",
        "Control ID": "IA-4",
        "Control Description": "Identifier Management - Manage information system identifiers for individuals requiring access."
    },
    {
        "Technique ID": "T1059",
        "Control ID": "SI-3",
        "Control Description": "Malicious Code Protection - Implement anti-malware protection capabilities at system entry and exit points."
    },
    {
        "Technique ID": "T1021.004",
        "Control ID": "AC-3",
        "Control Description": "Access Enforcement - Enforce approved authorizations for logical access to the system."
    }
]

# Convert to DataFrame for display
manual_nist_controls_df = pd.DataFrame(manual_nist_controls)

# Display the DataFrame
manual_nist_controls_df